In [50]:
import numpy as np
import random
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
import ipywidgets as widgets

symbols = ['B','.']
data_length = 30 # set it to >= 4

# generate random data
data = np.array([random.choice(symbols) for _ in range(data_length)])
B_count = np.sum(data == 'B')
print(data)

['.' '.' '.' 'B' 'B' '.' 'B' 'B' '.' 'B' '.' '.' 'B' 'B' 'B' 'B' 'B' '.'
 '.' 'B' 'B' '.' 'B' 'B' 'B' 'B' 'B' 'B' '.' '.']


In [51]:
def get_longest_streak(data):
    streaks = []
    i = 0
    max_score = 0
    while i < len(data):
        symbol = data[i]
        if symbol == 'B':
            score = 1
            start_pos = i
            while i < len(data):
                try:
                    if data[i+2] == 'B' and data[i+1] == '.':
                        score += 1
                        i += 2
                    else:
                        if score > max_score:
                            max_score = score
                        streaks.append({'score': score, 'start_pos': start_pos, 'end_pos': i})
                        break
                except IndexError:
                    if score > max_score:
                        max_score = score
                    streaks.append({'score': score, 'start_pos': start_pos, 'end_pos': i})
                    break
        i += 1

    # get all streaks with max_score
    max_score_streaks = [streak for streak in streaks if streak['score'] == max_score]
    
    for streak in max_score_streaks:
        # prevent zero length streaks
        if streak['start_pos'] == streak['end_pos']:
            streak['start_pos'] = 0
            streak['end_pos'] = -1


    return max_score_streaks
        
longest_streak = get_longest_streak(data)
print(longest_streak) 

[{'score': 2, 'start_pos': 4, 'end_pos': 6}, {'score': 2, 'start_pos': 7, 'end_pos': 9}, {'score': 2, 'start_pos': 20, 'end_pos': 22}]


In [52]:
def get_B_indices(data, start_pos=1, streak=None):
    # correct B's are left, incorrect B's are right
    # correct B means it is in correct position mathing start_pos(even,odd)
    B_indices = np.where(data == 'B')[0]
    if streak is not None:
        # remove B's in streak
        mask = np.logical_or(B_indices < streak['start_pos'], B_indices > streak['end_pos'])
        B_indices = B_indices[mask]

    even = start_pos % 2 == 0
    correct_Bs = B_indices[B_indices % 2 != even] # use correct B's if incorrent ran out

    incorrect_Bs = B_indices[B_indices % 2 == even] # use incorrect B's first

    return correct_Bs, incorrect_Bs

In [53]:
def validate(data, swaps):
    # add swaps for B's that are not in streak
    # B_indices is second value in each pair of swaps [[1st,2nd],[1st,2nd]]
    new_data = data.copy()
    # swap all the values
    for swap in swaps:
        new_data[swap[0]], new_data[swap[1]] = new_data[swap[1]], new_data[swap[0]]
    new_streak = get_longest_streak(new_data)[0]
    b_count = len(np.where(new_data == 'B')[0])
    if b_count == new_streak['score']:
        return
    correct_bs, incorrect_bs = get_B_indices(new_data, new_streak['start_pos'], new_streak)
    B_indices = np.concatenate((correct_bs, incorrect_bs))

    # left side
    for i in range (new_streak['start_pos'], 0, -2):
        if not len(B_indices):
            # no more B's to swap
            break

        if new_data[i] == '.':
            # dot is in incorrect position, so swap it
            swaps.append([i, B_indices[-1]])
            B_indices = np.delete(B_indices, -1)

    for i in range (new_streak['end_pos'], len(new_data), 2):
        if not len(B_indices):
            # no more B's to swap
            break

        if new_data[i] == '.':
            # dot is in incorrect position, so swap it
            swaps.append([i, B_indices[-1]])
            B_indices = np.delete(B_indices, -1)    

In [54]:
def process_default(data):

    B_indices = np.where(data == 'B')[0]

    new_data = data.copy()

    b_count = len(B_indices)
    if not b_count:
        # no B's on this side
        return 0
    solutions = []
    swaps = [] # indices of swaps
    start_positions = [B_indices[0], B_indices[0]+1] # find which is better, even or odd start position

    for orientation in range(2):
        for start_pos in start_positions:
            
            B_correct = np.concatenate((get_B_indices(new_data, start_pos=start_pos))) # get correct B's

            for i in range(start_pos, len(new_data), 2):

                if not len(B_correct):
                    # no more B's to swap
                    break
            
                if new_data[i] == 'B':
                    if i not in B_correct:
                        # skip because this B was used in swap
                        continue
                    # B in correct position, prevent it from swapping in future
                    B_correct = B_correct[B_correct != i]

                elif new_data[i] == '.':
                    # dot is in incorrect position, so swap it
                    # print('Swapping', i, 'with', B_correct[-1], 'to make a streak')
                    if orientation == 0:
                        swaps.append([i, B_correct[-1]])
                    else:

                        swaps.append([len(new_data)-i-1, len(new_data) - B_correct[-1]-1])

                    B_correct = np.delete(B_correct, -1)

            if len(B_correct):
                validate(data, swaps)
            solutions.append(swaps)
            swaps = []
        
        new_data = np.flip(new_data)
        B_indices = np.where(new_data == 'B')[0]
        start_positions = [B_indices[0], B_indices[0]+1]
    best_solution = min(solutions, key=lambda x: len(x))
    return best_solution
process_default(data)

[[8, 27], [10, 25], [18, 23], [28, 19], [2, 15]]

In [55]:
def swaps_to_streak(data):
    '''
    Input: np.array of side data, example: ['B', '.', '.', '.', '.', '.', 'B', 'B', '.']
           B_even: Bool, should B be placed in even positions like [0,2,4,6...]
    Output: number of swaps needed to continue the streak, for the example above it will be a 3, to make a streak like ['B', '.', 'B', '.', 'B', '.'...]
    '''
    B_count = np.sum(data == 'B')
    D_count = np.sum(data == '.')
    if D_count - B_count < 0 or B_count < 2:
        # it is impossible to sort, not enough space
        return -1, []
    
    swaps = []

    swaps.append(process_default(data))
    shortest_swaps = min(swaps, key=lambda x: len(x))

    print('Shortest swaps combination got from process_default is:', len(min(swaps, key=lambda x: len(x))))

    
    return len(shortest_swaps), shortest_swaps



# data = np.array(['.','.','B','B','.','B','.','B','.','.','.','B','.','.','.','.','.','.','.','.'])
# print(data)
# longest_streak = get_longest_streak(data)




In [56]:
def visualize(data, swaps, iter):
    '''
    Input: swaps - list of tuples with indices of elements to swap like (1,2)
    Output: None
    '''
    plt.figure()
    # plt.axis('off')
    # hide y axis
    plt.yticks([])
    plt.xticks([x for x in range(len(data))])
    # make size normal
    plt.xlim(-1, len(data))
    plt.ylim(-1, 1)
    visual_data = np.array(data, dtype=object)

    for i, symbol in enumerate(visual_data):
        # replace B with green, .(D) with blue
        if symbol == 'B':
            visual_data[i] = {'color': 'green', 'symbol': 'B'}
        else:
            visual_data[i] = {'color': 'blue', 'symbol': 'D'}

    if iter != 0:
        print("Iter:"   , iter)
        for i in range(iter):
            # swap and color last swap red
            if i == iter - 1:
                print("Swapping:", swaps[i])
                visual_data[swaps[i][0]]['color'] = 'red'
                visual_data[swaps[i][1]]['color'] = 'red'
            visual_data[swaps[i][0]], visual_data[swaps[i][1]] = visual_data[swaps[i][1]], visual_data[swaps[i][0]]
    else:
        print("Initial data")

    for i, symbol in enumerate(visual_data):
        # plot symbols
        plt.text(i, 0, symbol['symbol'], horizontalalignment='center', verticalalignment='center', color=symbol['color'])

    plt.show()


In [57]:
# data = np.array(['.','B','.','B','.','.','.','.','.','.','.','B','.','B','.','B']) # works
print('Current data:\n', data)
print("="*10, " RESULT ", "="*10)
swap_count, swaps = swaps_to_streak(data)
if(swap_count == -1):
    print('It is impossible to sort this data')
else:
    print('Swaps:', swaps)
    if swap_count < 40:
        # show visualization
        iter_visual =  interactive(visualize, data=widgets.fixed(data), swaps=widgets.fixed(swaps), iter=widgets.IntSlider(min=0, max=swap_count, step=1, value=0))
        display(iter_visual)




Current data:
 ['.' '.' '.' 'B' 'B' '.' 'B' 'B' '.' 'B' '.' '.' 'B' 'B' 'B' 'B' 'B' '.'
 '.' 'B' 'B' '.' 'B' 'B' 'B' 'B' 'B' 'B' '.' '.']
==========  RESULT  ==========
It is impossible to sort this data
